### Documentation for Google Earth Engine

Coding best practices
https://developers.google.com/earth-engine/guides/best_practices

API reference
https://developers.google.com/earth-engine/apidocs/

geemao library
https://geemap.org/foliumap/?h=addlayer#geemap.foliumap

geemap notebook examples
https://github.com/giswqs/geemap/tree/master/examples


## Precipitation + Evaporation layers -- NLDAS

Land Data Assimilation System (LDAS) combines multiple sources of observations (such as precipitation gauge data, satellite data, and radar precipitation measurements) to produce estimates of climatological properties at or near the Earth''s surface.

**This dataset is the primary (default) forcing file (File A) for Phase 2 of the North American Land Data Assimilation System (NLDAS-2). The data are in 1/8th-degree grid spacing; the temporal resolution is hourly.**




In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AX4XfWjF395AFOxNiTd29k8t34e6fDj7DRrBCveJ_5qOdnjc83QDBjceReM



Successfully saved authorization token.


In [ ]:
i_year = 2010
years_back = 10

# --- Does not use data from i_year --
# Initial date of interest (inclusive)
i_date = str(i_year - years_back)  + '-01-01' 
# Final date of interest (exclusive)
f_date = str(i_year) + '-01-01'

#print(i_date, f_date)

In [ ]:
nldas = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').filterDate(i_date,f_date)


evap = nldas.select('potential_evaporation')
print('evaporation filtered collection size:', evap.size().getInfo())

prec = nldas.select('total_precipitation')
print('precipitation filtered collection size:', prec.size().getInfo())

In [ ]:
aridity_index = prec.sum().divide( evap.sum().multiply(10))

In [ ]:
ca_bbox = ee.Geometry.Polygon([[[-124.409591,32], 
                               [-114.131211,32], 
                               [-114.131211,42.01], 
                               [-124.409591,42.01]]])

ca_aridity_index = aridity_index.clip(ca_bbox)

In [ ]:
task = ee.batch.Export.image.toDrive(ca_aridity_index,
                     description= str(i_year)+'aridity_index'
                     )

task.start()

In [ ]:
ca_aridity_index.projection().getInfo()

In [ ]:
max = ca_aridity_index.reduceRegion(reducer = ee.Reducer.max(),
                                 geometry = ca_bbox,
                                 scale=13915)
max.getInfo()

In [ ]:
aridity_index.projection().getInfo()
#evap[1].projection().getInfo()

In [ ]:
ca_total_evap = evap_nldas.sum().clip(ca_bbox)
ca_total_prec = prec_nldas.sum().clip(ca_bbox)

total_evap = evap_nldas.sum()
total_prec = prec_nldas.sum()



#ai = total_prec/total_evap



In [ ]:
import geemap

nldas_map = geemap.Map()

vis_evap = { 'min':0, 'max':3*8760, 'palette': purples}
vis_prec = { 'min':0, 'max':125*8760, 'palette': purples}

nldas_map.addLayer(total_evap,vis_evap, "total evaporation")
nldas_map.addLayer(total_prec,vis_prec, "total precipitation")

### Second attemp : calculate by month and then average

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [7]:
i_year = 2010
years_back = 10

# --- Does not use data from i_year --
# Initial date of interest (inclusive)
i_date = str(i_year - years_back)  + '-01-01' 
# Final date of interest (exclusive)
f_date = str(i_year) + '-01-01'

print(i_date, f_date)

2000-01-01 2010-01-01


In [3]:
nldas = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').filterDate(i_date,f_date)


evap = nldas.select('potential_evaporation')
print('evaporation filtered collection size:', evap.size().getInfo())

prec = nldas.select('total_precipitation')
print('precipitation filtered collection size:', prec.size().getInfo())

evaporation filtered collection size: 87672
precipitation filtered collection size: 87672


In [17]:
def string_date(month,year):
    if month == 12:
        return (str(year)+'-12-01',str(year+1)+'-01-01')    
    if month == 9:
        return (str(year)+'-09-01',str(year)+'-10-01')
    if month>9:
        return (str(year)+'-'+str(month)+'-01',str(year)+'-'+str(month+1)+'-01')
    return (str(year)+'-0'+str(month)+'-01',str(year)+'-0'+str(month+1)+'-01')
    
#for i in range(1,13):
#    print(string_date(i,2000)[0])

In [18]:
def month_arid_indx(month,year,evap,prec):
    date_param = string_date(month,year)
    m_evap = evap.filterDate(date_param[0],date_param[1])
    m_prec = prec.filterDate(date_param[0],date_param[1])
    return m_prec.sum().divide(m_evap.sum())

In [24]:
months_ai = []
for j in range(i_year-years_back,i_year):
    for i in range(1,13):
        #print(string_date(i,j))
        months_ai.append(month_arid_indx(i,j,evap,prec))


In [25]:
ai_coll = ee.ImageCollection(months_ai)
decade_ai = ai_coll.mean()

ee.imagecollection.ImageCollection

In [28]:
ca_bbox = ee.Geometry.Polygon([[[-124.409591,32], 
                               [-114.131211,32], 
                               [-114.131211,42.01], 
                               [-124.409591,42.01]]])

ca_aridity_index = decade_ai.clip(ca_bbox)

In [29]:
task = ee.batch.Export.image.toDrive(ca_aridity_index,
                     description= str(i_year)+'aridity_index'
                     )

task.start()

In [31]:
ca = ee.FeatureCollection('TIGER/2016/States').filter(ee.Filter.eq('NAME', 'California'))

In [34]:
import geemap

Map = geemap.Map()
Map.addLayer(ca)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…